# Labolatorium 6
## Rozwiązywanie układów równań liniowych

---
### Przydatne funkcje z których będziemy korzystać (z nimi głównie porównujemy nasze rozwiązania):
- `numpy.linalg.solve`
- `numpy.linalg.lstsq`
- `scipy.linalg.lu`

---
### Przydatne importy

In [8]:
import numpy as np
from numpy import linalg as npln
from scipy import linalg as scln
import time

---
### Zadanie 1 Metoda Gaussa-Jordana

Napisz i sprawdź funkcję rozwiązującą układ równań liniowych $n \times n$ metodą **Gaussa-Jordana**. Dla rozmiarów macierzy współczynników większych nż $500 \times 500$ porównaj czasy działania zaimplementowanej funkcji z czasami uzyskanymi dla wybranych funkcji bibliotecznych.

Ponieważ w zadaniu nie zostało wyszczególnione czy należy użyć `pivotingu` (i jeśli tak to częsciowego czy pełnego), moja implementacja zawiera pełny `pivoting`.

In [9]:
def gauss_Jordan(A, B):
    n = B.shape[0]
    result = np.concatenate((A, B), axis=1)
    
    #scalling so that max element in each row in A array is 1
    result = result/np.max(result[:, :-1], axis=1, keepdims=True)
    
    permutations = []
    # start pivoting
    for i in range(n):
        # find potential divider
        p_index = np.unravel_index(
            np.argmax(np.abs(result[i:, i:-1])), result[i:, i:-1].shape
        )
        # shift
        p_index = (p_index[0]+i, p_index[1]+i)
        permutations.append(p_index[1])

        # swap (permutate)
        result[[i, p_index[0]]] = result[[p_index[0], i]]
        result[:,[i, p_index[1]]] = result[:,[p_index[1], i]]

        for j in range(i + 1, n):
            tmp = result[j,i] / result[i,i]
            result[j] -= tmp * result[i]
            result[j,i] = 0.0

    
#     back substitution and division
    for i in range(n-1, -1, -1):
        for j in range(i-1, -1, -1):
            tmp = result[j,i] / result[i,i]
            result[j] -= tmp * result[i]
            result[j, i] = 0.0 
        result[i, -1] /= result[i, i]
        
    X = result[:, -1]
#     coming back to first order using saved permutations
    for i in range(n-1, -1, -1):
        p_index = permutations[i]
        X[[i,p_index]] = X[[p_index, i]]
    
    return X

#### Teraz czas przetestować daną funkcję.
W tym celu potrzebujemy danych wejściowych. Poniższa funkcja generuje losowy układ, dba jednak o to aby był on rozwiązywalny - $det(A) \neq 0$.

In [13]:
def generate_random_equations(size, min_val = -100.0 , max_val = 100.0):
    err = 10**(-8)
    A = np.random.uniform(low = min_val, high = max_val, size = (size, size))
    while np.linalg.det(A)<err:
        A = np.random.uniform(low = min_val, high = max_val, size = (size, size))
    B = np.random.uniform(low = min_val, high = max_val, size = (size, 1))
    return A, B

#### Należy również porównać czas wykonania różnych algorytmów i funkcji bibliotecznych
Poniższa funkcja mierzy czas wykonania danego algorytmu. Wypisuje ona zmierzony czas na standardowe wyjście i zwraca to, co zwrócił zadany algorytm.

In [14]:
def benchmark(algorithm, algorithm_name, **args):
    start = time.time()
    result = algorithm(**args)
    end = time.time()
    print(f"Algorithm {algorithm_name} took {end-start}s to execute")
    return result
   

#### Poniżej znajduje się porównanie czasów wykonania i poprawności wyników dla $n = 500, 750, 1000$

In [15]:
for n in [500, 750, 1000]:
    A, B = generate_random_equations(n)
    print(f"\n----------------------\n Macierz rozmiaru {n}x{n}\n")
    my_results = benchmark(gauss_Jordan, "Gauss Jordan", A = A, B = B)
    linalg_solve_results = benchmark(npln.solve, "numpy.linalg.solve", a = A, b = B)
    linalg_lstsq_results = benchmark(npln.lstsq, "numpy.linalg.lstsq", a = A, b = B)[0]
    
    print("\n Normy wektorów wynikowych (aby porównać poprawność rozwiązania): \n")
    print(f"Gauss Jordan - {npln.norm(my_results)}")
    print(f"numpy.linalg.solve - {npln.norm(linalg_solve_results)}")
    print(f"numpy.linalg.lstsq - {npln.norm(linalg_lstsq_results)}")


----------------------
 Macierz rozmiaru 500x500

Algorithm Gauss Jordan took 1.0235445499420166s to execute
Algorithm numpy.linalg.solve took 0.03497886657714844s to execute


/home/mikey/projects/mownit/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  This is separate from the ipykernel package so we can avoid doing imports until


Algorithm numpy.linalg.lstsq took 0.2889535427093506s to execute

 Normy wektorów wynikowych (aby porównać poprawność rozwiązania): 

Gauss Jordan - 29.086086492517783
numpy.linalg.solve - 29.086086492519104
numpy.linalg.lstsq - 29.086086492518728

----------------------
 Macierz rozmiaru 750x750

Algorithm Gauss Jordan took 2.697234630584717s to execute
Algorithm numpy.linalg.solve took 0.00914454460144043s to execute
Algorithm numpy.linalg.lstsq took 0.14055371284484863s to execute

 Normy wektorów wynikowych (aby porównać poprawność rozwiązania): 

Gauss Jordan - 17.296166133898794
numpy.linalg.solve - 17.296166133898
numpy.linalg.lstsq - 17.296166133897593

----------------------
 Macierz rozmiaru 1000x1000

Algorithm Gauss Jordan took 6.139344692230225s to execute
Algorithm numpy.linalg.solve took 0.018642425537109375s to execute
Algorithm numpy.linalg.lstsq took 0.5521142482757568s to execute

 Normy wektorów wynikowych (aby porównać poprawność rozwiązania): 

Gauss Jordan - 20.7

#### Analiza wyników

Jak widać, napisana przeze mnie metoda realizująca algorytm Gaussa Jordana jest zdecydowanie wolniejsza od funkcji bibliotecznych ( w tym zestawieniu wygrywa `numpy.linalg.solve`). Norma z wektora wynikowego, która posłużyła nam do sprawdzenia poprawności wyniku, jest równa wynikom obu funkcji bibliotecznych z dokładnością do około 10 miejsc po przecinku - co pozwala twierdzić, że `gauss_Jordan` działa poprawnie.

---
### Zadanie 2 Faktoryzacja LU

Napisz i sprawdź funkcję dokonującą faktoryzacji **A = LU** macierzy **A**. Zastosuj częściowe poszukiwanie elementu wiodącego oraz skalowanie.

Poniższa funkcja zwraca krotkę `(P, L, U)`, gdzie `P` to macierz permutacji, a `L` i `U` to wyniki faktoryzacji LU. Powyższe macierze są związane równością:

$ P \cdot A = L \cdot U$

In [26]:
def lu_factor(A):
    n = A.shape[0]
    P = np.identity(n)
    L = np.zeros((n, n))

    # scalling
    max_w = 1.0 / np.max(A, axis=0, keepdims=1)

    U = A * max_w.T
    
    for i in range(n):
        # half pivoting
        p_index = i + np.argmax(np.abs(A[i:, i]))
        U[[i, p_index]] = U[[p_index, i]]
        P[[i, p_index]] = P[[p_index, i]]
        L[[i, p_index]] = L[[p_index, i]]
        
        for j in range(i+1, n):
            factor = U[j,i] / U[i,i]
            L[j, i] = factor
            U[j] -= factor * U[i]
            U[j, i] = 0.0

    P = P * max_w
    L = L + np.identity(n)
            
    return P, L, U

### Czas przetestować wyżej powyższą funkcję

Zrobimy to jak poprzednio - porównamy czasy wykonania do funkcji bibliotecznej `scipy.linalg.lu` dla losowao wygenerowanych macierzy $n\times n$ ($n = 500, 750, 1000$). Sprawdzimy także poprawność zwracanych wyników (czy równanie $P \cdot A = L \cdot U$ jest spełnione) poprzez wyliczenie norm z danych macierzy.

In [32]:
for n in [500, 750, 1000]:
    A, _ = generate_random_equations(n)
    print(f"\n----------------------\n Macierz rozmiaru {n}x{n}\n")
    P, L, U = benchmark(lu_factor, "lu_factor", A = A)
    benchmark(scln.lu, "scipy.linalg.lu", a = A)
    
    print("\nNormy wektorów wynikowych (aby porównać poprawność rozwiązania): \n")
    print(f"P * A - {npln.norm(P @ A)}")
    print(f"L * U - {npln.norm(L @ U)}")


----------------------
 Macierz rozmiaru 500x500

Algorithm lu_factor took 0.5063714981079102s to execute
Algorithm scipy.linalg.lu took 0.01755666732788086s to execute

Normy wektorów wynikowych (aby porównać poprawność rozwiązania): 

P * A - 289.3951512500097
L * U - 289.39515125000963

----------------------
 Macierz rozmiaru 750x750

Algorithm lu_factor took 1.4112067222595215s to execute
Algorithm scipy.linalg.lu took 0.013455629348754883s to execute

Normy wektorów wynikowych (aby porównać poprawność rozwiązania): 

P * A - 433.9415925274443
L * U - 433.94159252744436

----------------------
 Macierz rozmiaru 1000x1000

Algorithm lu_factor took 2.314291477203369s to execute
Algorithm scipy.linalg.lu took 0.0224609375s to execute

Normy wektorów wynikowych (aby porównać poprawność rozwiązania): 

P * A - 578.2689817302606
L * U - 578.2689817302598


#### Analiza wyników

Tak jak można było się spodziewać - funkcja biblioteczna znów okazała się być zdecydowanie szybsza. Co do wyniku samej faktoryzacji można uznać ją za poprawną - normy z $P \cdot A$ oraz $L \cdot U$ są równe z dokładnością do  około kilkunastu miejsc poprzecinku.

---
### Zadanie 3 Analiza obwodu elektrycznego

Napisz program, który:

---
a) Wczytuje z pliku listę krawędzi grafu opisującego obwód elektryczny. Wagi krawędzi określają opór fragmentu obwodu między dwoma węzłami. Wierzchołki grafu identyfikowane są przez liczby naturalne.